## Ingest constructor.json file
Step 1- Read the json file using the spark dataframe reader

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StringType, IntegerType, DoubleType, StructField, TimestampType, DateType

In [0]:
constructors_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [0]:
constructors_df = spark.read\
    .schema(constructors_schema)\
    .json(f"{raw_folder_path}/{v_file_date}/constructors.json")

In [0]:
display(constructors_df)

In [0]:
constructors_df.printSchema()

#### Step 2 - Drop unwanted columns from the dataframe

In [0]:
from pyspark.sql.functions import col, current_timestamp, lit

In [0]:
constructor_dropped_df = constructors_df.drop(col("url"))

#### Step 3 - Rename cloumns and add ingestion date

In [0]:
constructor_final_df = constructor_dropped_df.withColumnRenamed("constructorId", "constructor_id")\
    .withColumnRenamed("constructorRef", "constructor_ref")\
    .withColumn("data_source", lit(v_data_source))\
     .withColumn("file_date", lit(v_file_date))
    

In [0]:
constructor_final_df = add_ingestion_date(constructor_final_df)

In [0]:
#constructor_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/constructors")
#constructor_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.constructors")
constructor_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.constructors")

In [0]:
%fs
ls /mnt/formula1dl2025practice/processed/constructors

In [0]:
dbutils.notebook.exit("Success")